### Arthur Henrique Dias

In [46]:
import pandas as pd
import numpy as np


df_receitas = pd.read_csv("/content/receitas.csv") #dados que serão utilizados durante a atividade

In [47]:
df_receitas #para olhar a estrutura da tabela receitas.csv antes de alteração

,Receita,Massa,Arroz Branco,Carne Bovina,Carne de Frango,Molho de Tomate,Queijo Muçarela,Caldo de Carne,Purê de Batata,Creme de Leite
0,Lasanha de Carne Moída,300.0,NaN,300.0,NaN,500.0,300.0,NaN,NaN,NaN
1,Macarrão à Bolonhesa,300.0,NaN,200.0,NaN,500.0,100.0,NaN,NaN,NaN
2,Arroz de Forno com Frango,NaN,400.0,NaN,300.0,400.0,200.0,NaN,NaN,NaN
3,Risoto de Carne,NaN,200.0,200.0,NaN,NaN,100.0,1000.0,NaN,NaN
4,Macarrão de Forno,300.0,NaN,NaN,300.0,500.0,300.0,NaN,NaN,200.0
5,Escondidinho de Carne,NaN,NaN,300.0,NaN,300.0,200.0,NaN,400.0,NaN
6,Arroz à Piemontese,NaN,400.0,NaN,NaN,NaN,150.0,NaN,NaN,240.0
7,Strogonoff de Frango,NaN,400.0,NaN,300.0,200.0,NaN,NaN,NaN,200.0
8,Pizza de Frigideira,300.0,NaN,150.0,NaN,200.0,150.0,NaN,NaN,NaN
9,Canelone de Frango,300.0,NaN,NaN,300.0,400.0,300.0,NaN,NaN,150.0


In [48]:


#Adicionar 2 receitas novas por código

nova_receita1 = {
    "Receita": "Frango Cremoso Fitness",
    "Massa": np.nan,
    "Arroz Branco": 200,
    "Carne Bovina": np.nan,
    "Carne de Frango": 300,
    "Molho de Tomate": 200,
    "Queijo Muçarela": 100,
    "Caldo de Carne": np.nan,
    "Purê de Batata": np.nan,
    "Creme de Leite": 100  # ingrediente novo
}

nova_receita2 = {
    "Receita": "Pure com carne de panela",
    "Massa": np.nan,
    "Arroz Branco": np.nan,
    "Carne Bovina": 250,
    "Carne de Frango": np.nan,
    "Molho de Tomate": 150,
    "Queijo Muçarela": 200,
    "Caldo de Carne": 500,
    "Purê de Batata": 300,
    "Creme de Leite": np.nan,
    "Cebola": 100   # ingrediente novo
}

df_receitas = pd.concat([df_receitas, pd.DataFrame([nova_receita1, nova_receita2])], ignore_index=True)


In [49]:
#Criar df_nutri


dados_nutri = {
    "Ingrediente": [
        "Massa",
        "Arroz Branco",
        "Carne Bovina",
        "Carne de Frango",
        "Molho de Tomate",
        "Queijo Muçarela",
        "Caldo de Carne",
        "Purê de Batata",
        "Creme de Leite"
    ],
    "kcal_100g": [
        220.10,  # Massa
        358.00,  # Arroz Branco
        153.00,  # Carne Bovina
        172.00,  # Carne de Frango
        45.05,   # Molho de Tomate
        318.00,  # Queijo Muçarela
        241.00,  # Caldo de carne
        115.16,  # Purê de Batata
        65.32    # Creme de Leite
    ]
}

df_nutri = pd.DataFrame(dados_nutri)

In [50]:

#Calcular peso total e kcal total das receitas

def calcular_kcal_receita(row):
    total_gramas = 0
    total_kcal = 0

    for ingrediente in df_nutri["Ingrediente"]:
        if ingrediente in row and not pd.isna(row[ingrediente]):
            qtd = row[ingrediente]  #valor em gramas
            kcal_100g = df_nutri.loc[df_nutri["Ingrediente"] == ingrediente, "kcal_100g"].values[0]

            total_gramas += qtd
            total_kcal += (qtd / 100) * kcal_100g

    return total_gramas, total_kcal

pesos = []
kcals = []

for idx, row in df_receitas.iterrows():
    peso, energia = calcular_kcal_receita(row)
    pesos.append(peso)
    kcals.append(energia)

df_receitas["Peso (g)"] = pesos
df_receitas["Valor Energético (kcal)"] = kcals

In [56]:
def calcular_eer(sexo, idade, peso, altura):

    s = sexo.strip().lower()

    # Formulas da tabela
    if s == "masculino":
        if 14 <= idade <= 18:
            eer = 19.12 + (3.68 * idade) + (8.62 * altura) + (20.28 * peso) + 20
        else:  # a partir de 19 anos
            eer = 581.47 - (10.83 * idade) + (8.30 * altura) + (14.94 * peso)
    else:  # feminino
        if 14 <= idade <= 18:
            eer = -297.54 - (22.25 * idade) + (12.77 * altura) + (14.73 * peso) + 20
        else:  # a partir de 19 anos
            eer = 575.77 - (7.01 * idade) + (6.60 * altura) + (12.14 * peso)

    return eer


In [52]:

#Indicar melhor receita

def indicar_receita(sexo, idade, peso, altura):
    eer = calcular_eer(sexo, idade, peso, altura)
    idx = (df_receitas["Valor Energético (kcal)"] - eer).abs().argmin() #encontra o índice da receita cujo valor energético é mais próximo da EER.
    return eer, df_receitas.loc[idx, "Receita"]




### Execução:


In [53]:
eer, receita = indicar_receita("masculino", 25, 60, 1.75) # sexo, idade, peso, altura
print("EER:", eer)
print("Receita recomendada:", receita)


EER: 1221.645
Receita recomendada: Pizza de Frigideira


In [54]:
df_receitas #com as alterações

,Receita,Massa,Arroz Branco,Carne Bovina,Carne de Frango,Molho de Tomate,Queijo Muçarela,Caldo de Carne,Purê de Batata,Creme de Leite,Cebola,Peso (g),Valor Energético (kcal)
0,Lasanha de Carne Moída,300.0,NaN,300.0,NaN,500.0,300.0,NaN,NaN,NaN,NaN,1400.0,2298.550
1,Macarrão à Bolonhesa,300.0,NaN,200.0,NaN,500.0,100.0,NaN,NaN,NaN,NaN,1100.0,1509.550
2,Arroz de Forno com Frango,NaN,400.0,NaN,300.0,400.0,200.0,NaN,NaN,NaN,NaN,1300.0,2764.200
3,Risoto de Carne,NaN,200.0,200.0,NaN,NaN,100.0,1000.0,NaN,NaN,NaN,1500.0,3750.000
4,Macarrão de Forno,300.0,NaN,NaN,300.0,500.0,300.0,NaN,NaN,200.0,NaN,1600.0,2486.190
5,Escondidinho de Carne,NaN,NaN,300.0,NaN,300.0,200.0,NaN,400.0,NaN,NaN,1200.0,1690.790
6,Arroz à Piemontese,NaN,400.0,NaN,NaN,NaN,150.0,NaN,NaN,240.0,NaN,790.0,2065.768
7,Strogonoff de Frango,NaN,400.0,NaN,300.0,200.0,NaN,NaN,NaN,200.0,NaN,1100.0,2168.740
8,Pizza de Frigideira,300.0,NaN,150.0,NaN,200.0,150.0,NaN,NaN,NaN,NaN,800.0,1456.900
9,Canelone de Frango,300.0,NaN,NaN,300.0,400.0,300.0,NaN,NaN,150.0,NaN,1450.0,2408.480
